#Install Libraries, Import Modules, Mount Drive

Install LibROSA, a library for working on audio data

In [ ]:
!pip install librosa

Import Modules

In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
from numpy.lib import stride_tricks

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Ensure there is a folder to store the data

In [ ]:
#if the designated folder does not exist, create it. Either way, set the spectrogram destination path to the specified folder.
if not os.path.isdir('/content/drive/My Drive/Colab Notebooks/Deep Learning/ChiroVox_Bat_Spectrograms_Folders/'):
  destination_base_path = os.mkdir('/content/drive/My Drive/Colab Notebooks/Deep Learning/ChiroVox_Bat_Spectrograms_Folders/')
else:
  destination_base_path = '/content/drive/My Drive/Colab Notebooks/Deep Learning/ChiroVox_Bat_Spectrograms_Folders/'

#Transform .wav files into Spectrograms

from https://stackoverflow.com/questions/44787437/how-to-convert-a-wav-file-to-a-spectrogram-in-python3 (2nd answer)

In [ ]:
""" short time fourier transform of audio signal """
def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))

    # zeros at beginning (thus center of 1st window should be for sample nr. 0)   
    samples = np.append(np.zeros(int(np.floor(frameSize/2.0))), sig)    
    # cols for windowing
    cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))

    frames = stride_tricks.as_strided(samples, shape=(int(cols), frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win

    return np.fft.rfft(frames)    

""" scale frequency axis logarithmically """    
def logscale_spec(spec, sr=44100, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))

    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):        
        if i == len(scale)-1:
            newspec[:,i] = np.sum(spec[:,int(scale[i]):], axis=1)
        else:        
            newspec[:,i] = np.sum(spec[:,int(scale[i]):int(scale[i+1])], axis=1)

    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[int(scale[i]):])]
        else:
            freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

    return newspec, freqs

""" plot spectrogram"""
def plotstft(audiopath, audio_num=None, binsize=2**10, plotpath=None, colormap="jet"):
    try:
      samplerate, samples = wav.read(audiopath)
    except:
      print('skipped:', audiopath)
      return

    s = stft(samples, binsize)

    sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)

    ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel

    timebins, freqbins = np.shape(ims)

    print("timebins: ", timebins)
    print("freqbins: ", freqbins)

    plt.figure(figsize=(15, 7.5))
    plt.imshow(np.transpose(ims), origin="lower", aspect="auto", cmap=colormap, interpolation="none")
    plt.colorbar()

    plt.xlabel("time (s)")
    plt.ylabel("frequency (hz)")
    plt.xlim([0, timebins-1])
    plt.ylim([0, freqbins])

    xlocs = np.float32(np.linspace(0, timebins-1, 5))
    plt.xticks(xlocs, ["%.02f" % l for l in ((xlocs*len(samples)/timebins)+(0.5*binsize))/samplerate])
    ylocs = np.int16(np.round(np.linspace(0, freqbins-1, 10)))
    plt.yticks(ylocs, ["%.02f" % freq[i] for i in ylocs])

    if plotpath:
        plt.savefig(os.path.join(plotpath, audio_num.split('.')[0]), bbox_inches="tight")
    else:
        plt.show()

    plt.clf()

    return ims

#re-iterate the base and destination paths we are getting the audio from
audio_base_path = '/content/drive/My Drive/Colab Notebooks/Deep Learning/ChiroVox_Bat_Data/' 
destination_base_path = '/content/drive/My Drive/Colab Notebooks/Deep Learning/ChiroVox_Bat_Spectrograms_Folders/'

#for each specie inside each genus, get teh audio file and plot it as a spectrogram using the above functions
#save the spectrograms to the designated destination genus folder
for genus_dir in os.listdir(audio_base_path):
  genus_path = os.path.join(audio_base_path, genus_dir)
  if os.path.isdir(genus_path):
    for specie_dir in os.listdir(genus_path):
      specie_path = os.path.join(genus_path, specie_dir)
      for audio_file in os.listdir(specie_path):
        if audio_file.endswith('.wav'):
          ims = plotstft(os.path.join(os.path.join(audio_base_path, specie_path), audio_file), audio_num = audio_file, plotpath = os.path.join(destination_base_path, genus_dir)) 


timebins:  2659
freqbins:  513
timebins:  1195
freqbins:  513
timebins:  5860
freqbins:  513


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: WavFileWarning: Chunk (non-data) not understood, skipping it.


timebins:  3219
freqbins:  513
timebins:  5860
freqbins:  513
timebins:  2634
freqbins:  513
timebins:  5860
freqbins:  513
timebins:  5860
freqbins:  513
timebins:  5860
freqbins:  513
timebins:  586
freqbins:  513
timebins:  1634
freqbins:  513
timebins:  545
freqbins:  513
timebins:  933
freqbins:  513
timebins:  3354
freqbins:  513
timebins:  1554
freqbins:  513
timebins:  1440
freqbins:  513


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: WavFileWarning: Reached EOF prematurely; finished at 2621440 bytes, expected 2621448 bytes from header.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log10


timebins:  2560
freqbins:  513


...

<Figure size 1080x540 with 0 Axes>

<Figure size 1080x540 with 0 Axes>

<Figure size 1080x540 with 0 Axes>

<Figure size 1080x540 with 0 Axes>

<Figure size 1080x540 with 0 Axes>

#Split the Spectrograms into Train, Validation, and Test Sets

Split the data stored in folders into train, validation, and test folders using https://github.com/jfilter/split-folders

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders

#use a ratio of 70% train, 10% validation, and 20% test
splitfolders.ratio("/content/drive/My Drive/Colab Notebooks/Deep Learning/ChiroVox_Bat_Spectrograms_Folders", output="/content/drive/My Drive/Colab Notebooks/Deep Learning/ChirVox_Bat_Spectrograms_Train_Test_Folders_More_Data",
    seed=1337, ratio=(.7, .1, .2), group_prefix=None, move=False)

Copying files: 2144 files [00:39, 54.38 files/s]
